# Automatic BSPF Eventplots

Creates automatic event plots based on catalog 

In [1]:
import os 
import obspy as obs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from tqdm.notebook import tqdm
from pprint import pprint

from functions.request_data import __request_data
from functions.add_distances_and_backazimuth import __add_distances_and_backazimuth
from functions.compute_adr_pfo import __compute_adr_pfo


In [2]:
if os.uname().nodename == 'lighthouse':
    root_path = '/home/andbro/'
    data_path = '/home/andbro/kilauea-data/'
    archive_path = '/home/andbro/freenas/'
elif os.uname().nodename == 'kilauea':
    root_path = '/home/brotzer/'
    data_path = '/import/kilauea-data/'
    archive_path = '/import/freenas-ffb-01-data/'

In [3]:
def __makeplot(config, st):

    
    st_in = st.copy()
    
    fig, ax = plt.subplots(6,1, figsize=(15,10), sharex=True)

    font = 14

    time_scaling, time_unit = 1, "sec"
    rot_scaling = 1e9
    trans_scaling = 1e6
    
    for i, tr in enumerate(st_in):
        
        if i in [0,1,2]:
            ax[i].set_ylabel(r"$\omega$ (nrad/s)", fontsize=font)
            ax[i].plot(tr.times()/time_scaling, tr.data*rot_scaling, 'k', label=tr.stats.station+"."+tr.stats.channel)

        elif i in [3,4,5]:
            ax[i].set_ylabel(r"u ($\mu$m/s)", fontsize=font)
            ax[i].plot(tr.times()/time_scaling, tr.data*trans_scaling, 'k', label=tr.stats.station+"."+tr.stats.channel)
            
        ax[i].legend(loc=1)
        
    ax[5].set_xlabel(f"Time ({time_unit}) from {st[0].stats.starttime.date} {str(st[0].stats.starttime.time).split('.')[0]} UTC", fontsize=font)
    ax[0].set_title(config['title']+f" | {config['fmin']} - {config['fmax']} Hz", fontsize=font, pad=10)
    
    plt.show();
    del st_in
    return fig

In [4]:
def __makeplotStreamSpectra2(st, config, fscale=None):

    from scipy import fftpack
    from andbro__fft import __fft
    import matplotlib.pyplot as plt

    st_in = st.copy()
    
    NN = len(st_in)
    rot_scaling, rot_unit = 1e9, r"nrad/s"
    trans_scaling, trans_unit = 1e6, r"$\mu$m/s"
        
    fig, axes = plt.subplots(NN,2,figsize=(15,int(NN*2)), sharex='col')

    font = 14
    
    plt.subplots_adjust(hspace=0.3)

    ## _______________________________________________

    st.sort(keys=['channel'], reverse=True)
    
    for i, tr in enumerate(st_in):

#         comp_fft = abs(fftpack.fft(tr.data))
#         ff       = fftpack.fftfreq(comp_fft.size, d=1/tr.stats.sampling_rate)
#         comp_fft = fftpack.fftshift(comp_fft)
#         ff, spec = ff[1:len(ff)//2], abs(fftpack.fft(tr.data)[1:len(ff)//2])

        if tr.stats.channel[-2] == "J":
            scaling = rot_scaling
        elif tr.stats.channel[-2] == "H":
            scaling = trans_scaling

        spec, ff, ph = __fft(tr.data*scaling, tr.stats.delta, window=None, normalize=None)
    
    
        ## _________________________________________________________________
        if tr.stats.channel[-2] == "J":
            axes[i,0].plot(
                        tr.times(),
                        tr.data*rot_scaling,
                        color='black',
                        label='{} {}'.format(tr.stats.station, tr.stats.channel),
                        lw=1.0,
                        )

        elif tr.stats.channel[-2] == "H":
            axes[i,0].plot(
                        tr.times(),
                        tr.data*trans_scaling,
                        color='black',
                        label='{} {}'.format(tr.stats.station, tr.stats.channel),
                        lw=1.0,
                        )
        ## _________________________________________________________________
        if fscale == "loglog":
            axes[i,1].loglog(ff, spec, color='black', lw=1.0)
        elif fscale == "loglin":
            axes[i,1].semilogx(ff, spec, color='black', lw=1.0)
        elif fscale == "linlog":
            axes[i,1].semilogy(ff, spec, color='black', lw=1.0)
        else:
            axes[i,1].plot(ff, spec, color='black', lw=1.0)         
        
        
        if tr.stats.channel[1] == "J":
            sym, unit = r"$\Omega$", rot_unit
        elif tr.stats.channel[1] == "H":
            sym, unit = "v", trans_unit
        else:
            unit = "Amplitude", "a.u."
            
        axes[i,0].set_ylabel(f'{sym} ({unit})',fontsize=font)    
        axes[i,1].set_ylabel(f'ASD \n({unit}/Hz)',fontsize=font)        
        axes[i,0].legend(loc='upper left',bbox_to_anchor=(0.8, 1.10), framealpha=1.0)
        
#         axes[i,0].ticklabel_format(axis='y', style='sci', scilimits=(0,0))
#         axes[i,1].ticklabel_format(axis='y', style='sci', scilimits=(0,0))

    if "fmin" in config.keys() and "fmax" in config.keys():
        axes[i,1].set_xlim(config['fmin'],config['fmax'])

    axes[NN-1,0].set_xlabel(f"Time from {tr.stats.starttime.date} {str(tr.stats.starttime.time)[:8]} (s)",fontsize=font)     
    axes[NN-1,1].set_xlabel(f"Frequency (Hz)",fontsize=font)     

    del st_in
    return fig

In [5]:
def __empty_stream(reference_stream):

    from numpy import ones
    from obspy import Stream, Trace
    
    t_ref = reference_stream[0]
    
    empty = Stream()

    for cha in ["BHZ", "BHN", "BHE"]:
        t = Trace()
        t.data = ones(len(t_ref))
        t.stats.sampling_rate = t_ref.stats.sampling_rate
        t.stats.starttime = t_ref.stats.starttime
        t.stats.network, t.stats.station, t.stats.channel = "PY", "RPFO", cha
        empty += t
        
    return empty

## Configurations

In [6]:
config = {}

## location of BSPF
config['BSPF_lon'] = -116.455439
config['BSPF_lat'] = 33.610643

## path for figures to store
config['outpath_figs'] = data_path+"BSPF/figures/triggered_all/"

## path for output data
config['outpath_data'] = data_path+"BSPF/data/waveforms/ACC/"

## blueSeis sensor (@200Hz)
config['seed_blueseis'] = "PY.BSPF..HJ*"

## Trillium 240 next to BlueSeis on Pier (@40Hz)
# config['seed_seismometer1'] = "II.PFO.10.BH*"
config['seed_seismometer1'] = "II.XPFO.30.BH*"

## STS2 next to BlueSeis (@200Hz)
config['seed_seismometer2'] = "PY.PFOIX..HH*"

config['path_to_catalog'] = data_path+"BSPF/data/catalogs/"

config['catalog'] = "BSPF_catalog_20221001_20230930_triggered.pkl"

## Event Info

In [7]:
events = pd.read_pickle(config['path_to_catalog']+config['catalog'])

In [8]:
events

,origin,latitude,longitude,depth,magnitude,type,seconds,trigger_time,arrival_time,event_time,cosum,backazimuth,distances_km
0,2022-10-01 05:36:11.240,33.489667,-116.507167,9120.0,1.13,ml,20171.24,2022-10-01T05:36:10.344538Z,2022-10-01T05:36:11.240000Z,2022-10-01T05:36:11.240000Z,4.0,199.712618,14.252022
1,2022-10-02 03:25:16.440,33.484333,-116.508500,8040.0,1.49,ml,98716.44,2022-10-02T03:25:19.269538Z,2022-10-02T03:25:16.440000Z,2022-10-02T03:25:16.440000Z,6.0,199.394033,14.850966
2,2022-10-02 15:51:37.180,33.480167,-116.398000,8240.0,1.68,ml,143497.18,2022-10-02T15:51:37.444539Z,2022-10-02T15:51:37.180000Z,2022-10-02T15:51:37.180000Z,6.0,159.749086,15.423592
3,2022-10-02 21:52:54.280,33.485333,-116.509333,9650.0,0.99,ml,165174.28,2022-10-02T21:52:57.394539Z,2022-10-02T21:52:54.280000Z,2022-10-02T21:52:54.280000Z,6.0,199.820062,14.772415
4,2022-10-04 22:33:48.370,33.489000,-116.504833,10060.0,1.00,ml,340428.37,2022-10-04T22:33:51.619539Z,2022-10-04T22:33:48.370000Z,2022-10-04T22:33:48.370000Z,6.0,198.791501,14.250442
...,...,...,...,...,...,...,...,...,...,...,...,...,...
223,2023-09-19 16:59:23.750,33.640333,-116.725833,14080.0,1.58,ml,30560363.75,2023-09-19T16:59:28.415000Z,2023-09-19T16:59:23.750000Z,2023-09-19T16:59:23.750000Z,6.0,277.552483,25.304601
224,2023-09-19 18:17:24.190,33.464333,-116.567167,9220.0,1.45,ml,30565044.19,2023-09-19T18:17:27.765000Z,2023-09-19T18:17:24.190000Z,2023-09-19T18:17:24.190000Z,4.0,212.629455,19.262266
225,2023-09-21 01:58:24.210,33.545833,-116.455000,4870.0,0.68,ml,30679104.21,2023-09-21T01:58:25.900000Z,2023-09-21T01:58:24.210000Z,2023-09-21T01:58:24.210000Z,6.0,179.675029,7.188470
226,2023-09-27 13:41:07.510,33.496500,-116.556833,13430.0,1.18,ml,31239667.51,2023-09-27T13:41:10.990000Z,2023-09-27T13:41:07.510000Z,2023-09-27T13:41:07.510000Z,6.0,216.668239,15.777807


## RUN LOOP

In [9]:
fails = 0
for jj, ev in enumerate(events):
    try:
        print(f"\n -> {jj} {events.origin[jj]} ")
    except:
        fails += 1
        print(f"failed {jj}")

print(fails)


 -> 0 2022-10-01 05:36:11.240000 

 -> 1 2022-10-02 03:25:16.440000 

 -> 2 2022-10-02 15:51:37.180000 

 -> 3 2022-10-02 21:52:54.280000 

 -> 4 2022-10-04 22:33:48.370000 

 -> 5 2022-10-05 15:39:32.640000 

 -> 6 2022-10-08 02:52:33.350000 

 -> 7 2022-10-08 02:59:39.290000 

 -> 8 2022-10-09 08:33:00.550000 

 -> 9 2022-10-09 11:44:41.790000 

 -> 10 2022-10-09 17:51:16.290000 

 -> 11 2022-10-14 03:49:02.940000 

 -> 12 2022-10-15 05:17:34.150000 
0


In [ ]:
global errors
errors = []

for jj, ev in enumerate(tqdm(events.index)):
# for jj, ev in enumerate(events[:2]):

    print(f"\n -> {jj} {events.origin[jj]} ")
    try:
        event_name = str(events.origin[jj]).replace("-","").replace(":","").replace(" ", "_").split(".")[0]
    except:
        print(f" -> {jj}: error for event.origin")
        continue

    filename=config['outpath_figs']+"raw/"+f"{event_name}_raw.png"

    ## check if file already exists
#     if os.path.isfile(filename):
#         print(f" -> file alread exits for {event_name}")
#         continue

    ## configuration adjustments
    config['title'] = f"{events.origin[jj]} UTC | M{events.magnitude[jj]}"
    config['tbeg'] = obs.UTCDateTime(str(events.origin[jj]))-30


    ## select appropriate seismometer
    if config['tbeg'].date < obs.UTCDateTime("2023-04-01"):
        config['seed_seismometer'] = config['seed_seismometer1']
        config['fmin'], config['fmax'] = 0.01, 18.0
    else:
        config['seed_seismometer'] = config['seed_seismometer2']
        config['fmin'], config['fmax'] = 0.01, 80.0


    ## same endtime for all
    config['tend'] = obs.UTCDateTime(events.origin[jj])+150


    ## load and process blueSeis data
    try:
        py_bspf0, py_bspf_inv = __request_data(config['seed_blueseis'], config['tbeg'], config['tend'])

    except Exception as e:
        print(e)
        print(f" -> failed to request BSPF for event: {ev}")
        errors.append(f" -> failed to request BSPF for event: {ev}")
        continue

    ## load and process seismometer data
    try:
        ii_pfo0, ii_pfo_inv = __request_data(config['seed_seismometer'], config['tbeg'], config['tend'], translation_type="ACC")

    except Exception as e:
        print(e)
        print(f" -> failed to request BSPF for event: {ev}")
        continue

    if py_bspf0 is None or ii_pfo0 is None:
        continue

    ## processing data
    if ii_pfo0[0].stats.sampling_rate != py_bspf0[0].stats.sampling_rate:
        py_bspf0.resample(ii_pfo0[0].stats.sampling_rate)


    ## joining data
    st0 = py_bspf0
    st0 += ii_pfo0


    ## compute ADR

    ## for complete array
    try:
        pfo_adr = __compute_adr_pfo(config['tbeg'], config['tend'], submask="all")
        for tr in pfo_adr:
            tr.stats.location = "all"
        st0 += pfo_adr
    except:
        print(" -> failed to compute ADR all ...")
        pfo_adr = __empty_stream(st0)

    ## for mid array
    try:
        pfo_adr = __compute_adr_pfo(config['tbeg'], config['tend'], submask="mid")
        for tr in pfo_adr:
            tr.stats.location = "mid"
        st0 += pfo_adr
    except:
        print(" -> failed to compute ADR mid ...")
        pfo_adr = __empty_stream(st0)

    ## for inner array
    try:
        pfo_adr = __compute_adr_pfo(config['tbeg'], config['tend'], submask="inner")
        for tr in pfo_adr:
            tr.stats.location = "inn"
        st0 += pfo_adr
    except:
        print(" -> failed to compute ADR inner ...")
        pfo_adr = __empty_stream(st0)

    st0 = st0.sort()

    ## processing data stream
    st = st0.copy();
    st.detrend("linear");
    st.taper(0.01);
    st.filter("bandpass", freqmin=config['fmin'], freqmax=config['fmax'], corners=4, zerophase=True);


    st.trim(config['tbeg'], config['tend']);
    st0.trim(config['tbeg'], config['tend']);

    print(st0)

    ## store waveform data
    num = str(jj).rjust(3,"0")
    waveform_filename = f"{num}_{str(events.origin[jj]).split('.')[0].replace('-','').replace(':','').replace(' ','_')}.mseed"
    st0.write(config['outpath_data']+waveform_filename, format="MSEED");


    ## create eventname
    # event_name = str(events.origin[jj]).replace("-","").replace(":","").replace(" ", "_").split(".")[0]


    ## plotting figures    
    fig1 = st0.plot(equal_scale=False, show=True);

#     fig2 = __makeplot(config, st)

#     fig3 = __makeplotStreamSpectra2(st, config, fscale="linlin");

    ## saving figures
    fig1.savefig(config['outpath_figs']+"raw/"+f"{event_name}_raw.png", dpi=200, bbox_inches='tight', pad_inches=0.05)

#     fig2.savefig(config['outpath_figs']+"filtered/"+f"{event_name}_filtered.png", dpi=200, bbox_inches='tight', pad_inches=0.05)

#     fig3.savefig(config['outpath_figs']+"spectra/"+f"{event_name}_spectra.png", dpi=200, bbox_inches='tight', pad_inches=0.05)


## End of File

In [ ]:
# waveform_filename = f"{num}_{str(events.origin[jj]).split('.')[0].replace('-','').replace(':','').replace(' ','_')}.mseed"
events.origin[1]

In [ ]:
st0.plot()

## Testing Signal-to-Noise ratios

In [ ]:
# from numpy import nanmean, sqrt

# win_length_sec = 10 ## seconds

# t_trigger = events.trigger_time[jj]
# t_rel_sec = t_trigger-config['tbeg']
# fig, ax = plt.subplots(len(st0),1, figsize=(15,15))

# for i, tr in enumerate(st0):

#     df = tr.stats.sampling_rate 

#     NN = int(df * win_length_sec) ## samples

#     t_rel_spl = t_rel_sec*df ## samples

#     t_offset = df * 2 ## samples

#     noise = nanmean(tr.data[int(t_rel_spl-NN):int(t_rel_spl)]**2)
#     signal = nanmean(tr.data[int(t_rel_spl):int(t_rel_spl+NN)]**2)

#     SNR = sqrt(signal/noise)



#     ax[i].plot(tr.data)

#     ax[i].axvline(t_rel_spl, color="red")
#     ax[i].axvline(t_rel_spl+NN, color="red")

#     ax[i].axvline(t_rel_spl, color="g")
#     ax[i].axvline(t_rel_spl-NN, color="g")

#     print(SNR)

# plt.show();
